<a id='Top'></a>
# Python - Data cleaning

We'll perform data cleaning on a real-world data set of artworks contained in the Museum of Modern Art (MoMA). MoMA, a museum in New York City, has one of the largest collections of modern art in the world.

In this exercise, we parse numbers from complex string and exercise differnt methods in manupuating strings.

*This exercise is a part of Dataquest.io class of "Python for Data Science: Fundamentals."*

*The dataset can be found at: 
https://www.kaggle.com/momanyc/museum-collection*

### Below is the explanation of each column:
- Title: The title of the artwork.
- Artist: The name of the artist who created the artwork.
- Nationality: The nationality of the artist.
- BeginDate: The year in which the artist was born.
- EndDate: The year in which the artist died.
- Gender: The gender of the artist.
- Date: The date that the artwork was created.
- Department: The department inside MoMA to which the artwork belongs.

**Jump to different sections:**
[Process Gender and Nationality columns](#gender_nation)

[Process data from Begin/EndDate column](#begin_end_date)

[Process data from the Date column](#date)

[Calculate the age where the artist create the art](#age)

In [ ]:
open_file = open('/Users/spare/Downloads/projects_python/Artworks.csv')

from csv import reader
moma_file = reader(open_file)
moma = list(moma_file)

num_rows = len(moma)
print('Number of row in the record:', num_rows)
print('\nPrint of first couple rows:\n')
print(moma[0],'\n')
print(moma[1])

#Limit the size of the data
moma=moma[1:2000]
#moma=moma[1:]

<a id='gender_nation'></a>
## Process Gender and Nationality columns
- Remove "(" and ")" from Nationality column (index 4) and "Gender" (index 7).
- Use str.title( ) to format the columns.
- Using string.replace()
- Assign value to empty string for the 2 columns. 

In [ ]:
for row in moma:
    gender = str(row[7])
    gender = gender.title()
    nationality = str(row[4])
    nationality = nationality.title()

    if gender == "":
        gender = "Gender Unknown/Other"
    row[7]=gender
    
    if nationality == "":
        nationality = "Nationality Unknown"
    row[4]=nationality

<a id='begin_end_date'></a>
[Back to Top](#Top)

## Process BeginDate and EndDate columns
We'll now look at the BeginDate and EndDate columns. These columns contain the birth and death dates of the artist who created the work.
- Remove the parentheses from the start and the end of each value.
- Convert the values from the string type to an integer type. This will help us perform calculations with them later.

In [ ]:
test_data = ["-1841", "(1932) (0)","(1944) (1945)","(1931) (1945)   (0)", 1983]
bad_char = ["-", "(", ")"]
new_test_data=[]

def process_begin_end_date(a_str):
    #first convert input to string.
    if isinstance(a_str,(int)):
        a_str = str(a_str)
    
    new_a_str = []
#     print('String to process:',a_str)
    for bad in bad_char:
        if bad in a_str:
            a_str = a_str.replace(bad," ")
        
    a_str=a_str.split(" ")
    #print(a_str)
    for index in a_str:
        if index != "":
            new_a_str.append(int(index))
#             print(index)
                
    return new_a_str

#For testing
#a = process_begin_end_date("(1932) (0)")
#print(a)

for ele in test_data:
    new_ele = process_begin_end_date(ele)
    new_test_data.append(new_ele)

print(new_test_data)
print(type(new_test_data[0][0]))
    

<a id='date'></a>
[Back to Top](#Top)
## Process data from the Date column
- The strip_characters() function to remove the bad characters and use it on test_data.
- The "bad characters" are described in bad_chars. 
    - These are the characters that will be stripped out of the string. 
- The bad_string is used to described strings that are not allowable. 

In [ ]:
def strip_characters(a_str):
    
    #Convert input to a string
    if not isinstance(a_str,(str)):
        #print('convert to string:', a_str)
        a_str = str(a_str)
    
    a_str = a_str.lower() #convert the string to lower case
    
    for char in bad_chars: #replace bar_chars character in the string with ""
        a_str = a_str.replace(char,"")
            
    for ele in bad_string: #string that have unallowable string will be 0
        if ele in a_str:
            #print('ele: ', ele,'in string: ', a_str)
            a_str = '0'        
    
    return a_str

bad_chars = ["(",")","c","C", ".","s","'", " ", ";"]
bad_string = ["unknown", "n", "b", "a", "e", "l", 
              "model", "variou", "moow", "?", "ot", "/" ]

test_data = ["1912", "1929", "1913-1923",
             "(1951)", "1994", "1934",
             "c. 1915", "1995", "c. 1912",
             "(1988)", "2002", "1957-1959",
             "c. 1955.", "c. 1970's", 
             "C. 1990-1999", 
             "Unknown", "n.d.", 1983, "2001–2002", 
             "before 1933", "after 2001", "early 1132", "Early 1950",
             "1933,model1987", "a 1986",
             "1976-77", ".1-3 1987; .4 1990" ]

print('test_data=\n',test_data)
#***Becareful, there is different between – and - 
#   Different ASCII, 45 and 8211.

stripped_test_data =[]

for ele in test_data:
    #print(type(ele))
    new_string = strip_characters(ele)
    stripped_test_data.append(new_string)

print('\nstripped_test_data=\n',stripped_test_data)

**There are two different scenarios that we need to cater for when converting these to integers:**
- Some are a single year, e.g. 1912.
- Some are ranges of years, e.g. 1913-1923.

Here are the ways we'll treat the various cases:
- Where there is a single year, we'll keep it.
- Where there is a year range, we'll average the two years.

We want to write code that does the following for each value:
- Checks if the dash character (-) is in the string so we know if it's a range or not.
- If the date is a range:
    - Splits the string into two strings, before and after the dash character.
    - Converts the two numbers to the integer type and then average them by adding them together and dividing by two.
    - Uses the round() function to round the average, so values like 1964.5 become 1964.
- If the date isn't a range:
    - Converts the value to an integer type.

In [ ]:
def process_date(some_string):
    #Assume some_string is already a string because strip_characters function 
    # will be executed first.
    
    new_string=0
    if "-" in some_string: #ascii 45
        split_string = some_string.split("-")
        split_string[0] = int(split_string[0])
        split_string[1] = int(split_string[1])
        if 3000>split_string[1]> 1000: #for "1976-77"
            new_string = round((split_string[0]+split_string[1])/2)
        else:
            new_string = 0
    
    elif "–" in some_string: #ascii 8211
        split_string = some_string.split("–")
        split_string[0] = int(split_string[0])
        split_string[1] = int(split_string[1])
        if 3000>split_string[1] > 1000: #for "1976-77"
            new_string = round((split_string[0]+split_string[1])/2)
        else:
            new_string = 0
        
    elif ";" in some_string:
        split_string = some_string.split(";")
        split_string[0] = int(split_string[0])
        split_string[1] = int(split_string[1])
        if 3000> split_string[1] > 1000: #for "1976-77"
            new_string = round((split_string[0]+split_string[1])/2)
        else:
            new_string = 0
    elif some_string == "":
        new_string =0
    
    return new_string

#### Testing process_date with stripped_test_data

In [ ]:
print('Current test data (before):')
print(stripped_test_data)

processed_test_data=[]

for ele in stripped_test_data:
    date = process_date(ele)
    processed_test_data.append(date)

print('\nNew test data (after):')
print(processed_test_data)

[Back to Top](#Top)

[Back to Process data from the Date column](#date)
### Execute on actual Moma data for:

BeginDate and EndDate column (index 5 & 6)
- process_begin_end_date function.

Date column (index 8)
- strip_characters function.
- process_date function.

The resulted data on these column are integer or list of integers.

In [ ]:
cnt=0
for row in moma:
    cnt+=1
    line=cnt
    title = row[0]
    obj_id = row[17]
    
    begin_date = process_begin_end_date(row[5])
#     print(row[5])
    row[5] = begin_date
    
    end_date = process_begin_end_date(row[6])
    row[6] = end_date
    
    date = strip_characters(row[8])
    date = process_date(date)
    if date > 2020:
        print('WHAT!!! Title=',title)
        print('line:',line,'object_id:', obj_id, 'date:',date)
    
    row[8] = date

Print out some sample data

In [ ]:
for row in moma[0:10]:
    print(row[0],row[5], row[6], row[8])

[Back to Top](#Top)

[Back to Process data from Begin/EndDate column](#begin_end_date)

<a id='age'></a>

## Calculate the age where the artist create the art
- Age = Date - BeginDate.
- BeginDate is a list (of 'int') becuase there are potentially multiple artists.


In [ ]:
ages=[]
age =[]

for row in moma:
    title = row[0] 
    date  = row[8] #Date 
    birth = row[5] #BeginDate
    age=[]
    
    if date!=0:
#         print('date=', date)
#         print('birth=', birth)
        for ele in birth:
            if ele !=0: 
                ele=date-ele
                #print(ele)
            else:
                ele=0
            age.append(ele)
            #print(age)
    
    ages.append(age)

# print('ages list=',ages)


[Back to Top](#Top)